# Prototype
[*v0.1*]

Prototyping with TMDB.

# Setup

Cells in this section handle notebook setup, like importing packages and functions/vars from scripts.

## Imports

Import `stdlib` packages (i.e. `pathlib.Path`) and package dependencies.

### stdlib

In [1]:
from pathlib import Path
import json
from typing import Any, Optional, Union
import random

### Custom modules & vars

In [2]:
## Constants
from lib.constants import (
    auth_endpoint,
    movie_endpoint,
    token_endpoint,
    api_key,
    base_url,
    tv_endpoint,
    session_endpoint,
    popular_tv_endpoint,
    basic_auth_headers,
)

In [3]:
from utils.time_utils import benchmark
from utils.file_utils import check_file_exist

In [4]:
from core.config import api_settings, app_settings, logging_settings
from utils.logger import get_logger

In [5]:
from utils.tmdb_utils import (
    authenticate,
    get_popular_tv,
    get_request_token,
    retrieve_token,
    get_bad_ids,
    append_bad_id,
    check_bad_id,
    generate_rand_id,
    get_tv_episode,
)

In [6]:
from core.db import Base, create_base_metadata, get_engine, get_session
from domain.schemas.tmdb import tmdb_media_schemas, tmdb_responses

### Dependencies

Packages installed with `pip` (or some equivalent tool)

In [7]:
import httpx

## Global Vars

Variables for use throughout the notebook

In [8]:
nb_log: bool = True
nb_verbose: bool = False

In [9]:
log = get_logger(__name__, level="INFO")

In [10]:
engine = get_engine(connection="db/demo.sqlite", echo=True)
SessionLocal = get_session(engine=engine)
create_base_metadata(base_obj=Base, engine=engine)

2023-06-19 17:51:05,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 17:51:05,030 INFO sqlalchemy.engine.Engine COMMIT


True

## Functions

Notebook-level functions. These differ from functions imported from scripts in that they are either prototypes, or functions meant only for the notebook.

### Notebook Functions

### Prototype

## Classes

Notebook-level classes. These differ from classes/models imported from scripts in that they are either prototypes, or functions meant only for the notebook.

# Operations

Functions & data operations.

In [11]:
## Test authentication
_auth = authenticate()
display(f"Auth response: [{_auth.status_code}: {_auth.reason_phrase}]")

[INFO][2023-06-19_17:51:05][utils.tmdb_utils][authenticate ln: 36]: Requesting https://api.themoviedb.org/3/authentication


'Auth response: [200: OK]'

In [12]:
## Test retrieving auth token
_token = get_request_token()
display(f"Token response: [{_token.status_code}: {_auth.reason_phrase}]")

## Retrieve token string from resonse
token = retrieve_token(_token.text)
display(f"Token ({type(token)}): {token}")

[INFO][2023-06-19_17:51:05][utils.tmdb_utils][get_request_token ln: 90]: Requesting https://api.themoviedb.org/3/authentication/token/new


'Token response: [200: OK]'

"Token (<class 'domain.schemas.tmdb.tmdb_responses.ReqToken'>): success=True expires_at='2023-06-19 22:51:05 UTC' request_token='876431024aceca759e1c2cb257a07d175d3d2584'"

## Examples

### Example responses

In [13]:
examples_dir: str = "examples"
example_responses_dir: str = "responses"
ex_res_path: str = f"{examples_dir}/{example_responses_dir}"

if not Path(ex_res_path).exists():
    Path(ex_res_path).mkdir(exist_ok=True, parents=True)

In [14]:
ex_movie_res_file: str = f"{ex_res_path}/ex_movie_response.json"
ex_tv_res_file: str = f"{ex_res_path}/ex_tvshow_response.json"

In [15]:
with open(ex_movie_res_file, "r+") as movie_res_file:
    ex_movie_res: dict = json.loads(movie_res_file.read())

with open(ex_tv_res_file, "r+") as tv_res_file:
    ex_tv_res: dict = json.loads(tv_res_file.read())

In [16]:
ex_movie_keys = ex_movie_res.keys()
# display(f"Movie dict keys:")

# for k in ex_movie_keys:
#     display(f"Key [{k}] ({type(ex_movie_res[k]).__name__}): {ex_movie_res[k]}")

In [17]:
ex_tv_keys = ex_tv_res.keys()
# display(f"TV dict keys:")

# for k in ex_tv_keys:

In [18]:
shared_keys: list[str] = []
movie_only_keys: list[str] = []
tv_only_keys: list[str] = []

In [19]:
for k in ex_movie_keys:
    if not k in ex_tv_keys:
        # display(f"Movie-specific key found: {k} ({type(ex_movie_res[k]).__name__})")
        movie_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [20]:
for k in ex_tv_keys:
    if not k in ex_movie_keys:
        # display(f"TV-specific key found: {k} ({type(ex_tv_res[k]).__name__})")

        tv_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [21]:
display(f"Keys only in movie response:")

for movie_k in movie_only_keys:
    display(movie_k)

'Keys only in movie response:'

'belongs_to_collection'

'budget'

'imdb_id'

'original_title'

'release_date'

'revenue'

'runtime'

'title'

'video'

In [22]:
display(f"Keys only in tv response:")

for tv_k in tv_only_keys:
    display(tv_k)

'Keys only in tv response:'

'created_by'

'episode_run_time'

'first_air_date'

'in_production'

...

'number_of_seasons'

'origin_country'

'original_name'

'seasons'

'type'

In [23]:
display(f"Keys in both movies and tv shows:")

display(shared_keys)

'Keys in both movies and tv shows:'

['adult',
 'backdrop_path',
 'genres',
 'homepage',
 'id',
 'original_language',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'spoken_languages',
 'status',
 'tagline',
 'vote_average',
 'vote_count']

### TV Show examples

In [24]:
## Retrieve popular TV shows
popular_tv = get_popular_tv()
display(
    f"Retrieve popular TV shows response: [{popular_tv.status_code}: {popular_tv.reason_phrase}]"
)

[INFO][2023-06-19_17:51:05][utils.tmdb_utils][get_popular_tv ln: 222]: Requesting https://api.themoviedb.org/3/tv/popular?language=en-US&page=1


'Retrieve popular TV shows response: [200: OK]'

In [25]:
## Create dictionary from popular_tv response
pop_tv_dict: dict = popular_tv.text_json()
display(pop_tv_dict.keys())

display(pop_tv_dict["results"][0])

display(f"genre_ids ({type(pop_tv_dict['results'][0]['genre_ids'])})")

## Create MediaResponse class instance from pop_tv_dict
pop_tv: tmdb_media_schemas.MediaResponse = tmdb_media_schemas.MediaResponse.parse_obj(
    pop_tv_dict
)

# display(f"Popular TV shows response type: ({type(pop_tv)}): {pop_tv}")

dict_keys(['page', 'results', 'total_pages', 'total_results'])

{'backdrop_path': '/t2rAdgjSh0WYbXzdOB5zTDqzdCI.jpg',
 'first_air_date': '2022-11-02',
 'genre_ids': [18],
 'id': 213713,
 'name': 'Faltu',
 'origin_country': ['IN'],
 'original_language': 'hi',
 'original_name': 'Faltu',
 'overview': "What's in a name? Amidst the arid landscape of Rajasthan, a young woman with dreamy eyes struggles to prove her worth.",
 'popularity': 2699.206,
 'poster_path': '/lgyFuoXs7GvKJN0mNm7z7OMOFuZ.jpg',
 'vote_average': 4.6,
 'vote_count': 29}

"genre_ids (<class 'list'>)"

In [26]:
## grab a random sample TV show from popular TV shows
pop_tv_results: int = len(pop_tv.results)
rand_pop_tv: int = random.randint(0, pop_tv_results - 1)

_sample: tmdb_media_schemas.MediaTVShow = pop_tv.results[rand_pop_tv]
display(f"Sample ({type(_sample)}): {_sample}")

"Sample (<class 'domain.schemas.tmdb.tmdb_media_schemas.MediaTVShow'>): adult=None backdrop_path='/4NcAz1QIqYnhe3u2pnVEVNwfTZf.jpg' genres=None genre_ids=[18, 10766, 10751] overview='Virat sacrifices his love to honour the promise he made to a dying man. Trapped between the past and the present, will he find love beyond the chains of duty?' popularity=2207.022 poster_path='/u3NVGYCpkAgBArXogLuHPfpSNwG.jpg' tmdb_id=111453 original_language='hi' vote_average=5.4 vote_count=41 homepage=None production_companies=None runtime=None created_by=None episode_run_time=None first_air_date='2020-10-05' in_production=None languages=None last_air_date=None last_episode_to_air=None next_episode_to_air=None name='Ghum Hai Kisikey Pyaar Meiin' origin_country=['IN'] original_name='घुम है किसिकी प्यार में' networks=None number_of_episodes=None number_of_seasons=None seasons=None type=None"

In [27]:
## Retrieve full listing from TMDB
url = f"{api_settings.BASE_URL}/tv/{_sample.tmdb_id}"
display(f"Sample TV URL: {url}")

'Sample TV URL: https://api.themoviedb.org/3/tv/111453'

In [28]:
_show = get_tv_episode(tmdb_id=_sample.tmdb_id)
_show_dict: dict = _show.text_json()

tv_show: tmdb_media_schemas.MediaTVShow = tmdb_media_schemas.MediaTVShow.parse_obj(
    _show_dict
)

[INFO][2023-06-19_17:51:05][utils.tmdb_utils][get_tv_episode ln: 274]: Requesting https://api.themoviedb.org/3/tv/111453


In [29]:
display(f"Sample TV Show ({type(tv_show).__name__})")

display(tv_show)

'Sample TV Show (MediaTVShow)'

MediaTVShow(adult=False, backdrop_path='/qcpC9lv6VLL4Zw45EveYELyje1w.jpg', genres=[MediaGenres(id=18, name='Drama'), MediaGenres(id=10766, name='Soap'), MediaGenres(id=10751, name='Family')], genre_ids=None, overview='Virat sacrifices his love to honour the promise he made to a dying man. Trapped between the past and the present, will he find love beyond the chains of duty?', popularity=2207.022, poster_path='/u3NVGYCpkAgBArXogLuHPfpSNwG.jpg', tmdb_id=111453, original_language='hi', vote_average=5.378, vote_count=41, homepage='', production_companies=[ProductionCompanies(id=141735, logo_path='/ba2GSbH2KnInGel6L6XIy3dtGQ5.png', name='Cockcrow Entertainment & Shaika Films', origin_country='IN')], runtime=None, created_by=[TVShowCreator(id=1846290, credit_id='5f8476a169eb900038be37e3', name='Leena Gangopadhyay', gender=1, profile_path='/zHaUe3Lckgsm12xNtqMq99jPnyO.jpg')], episode_run_time=[22], first_air_date='2020-10-05', in_production=True, languages=['hi'], last_air_date='2023-06-18', 

### Random movie

In [30]:
movie_objs: list[tmdb_media_schemas.MediaMovie] = []
rand_movie = generate_rand_id(type="movie")

display(f"Random movie ID: {rand_movie}")

'Random movie ID: 159'

In [31]:
url = f"https://api.themoviedb.org/3/movie/{rand_movie}?language=en-US"
display(f"URL: {url}")

'URL: https://api.themoviedb.org/3/movie/159?language=en-US'

In [32]:
## Get random movie
try:
    with httpx.Client(headers=basic_auth_headers) as client:
        res = client.get(url)

        if not res.status_code == 200:
            display(
                f"Non-200 response returned: [{res.status_code}: {res.reason_phrase}]: {res.text}"
            )

            display(f"Adding ID {rand_movie} to list of known bad IDs")

            append_bad_id(bad_id=rand_movie, bad_id_file="bad_movie_ids")

        results: tmdb_responses.ReqResponse = tmdb_responses.ReqResponse.parse_obj(
            res.__dict__
        )

        res_json: dict = json.loads(res.text)

        _movie: tmdb_media_schemas.MediaMovie = tmdb_media_schemas.MediaMovie.parse_obj(
            res_json
        )

        display(f"Movie: {_movie}")

        movie_objs.append(_movie)

except Exception as exc:
    raise Exception(
        f"Unhandled exception requesting movie with ID: {rand_movie}. Details: {exc}"
    )

"Movie: adult=False backdrop_path='/1BMnXAAMbGN8N1xm2ZvlSVVUQiy.jpg' genres=[MediaGenres(id=35, name='Comedy'), MediaGenres(id=18, name='Drama')] genre_ids=None overview='Der Bewegte Mann is a German comedy about a heterosexual man, Axel, who is thrown out of his girlfriends home for cheating and ends up moving in with a gay man. Axel learns the advantages of living with gay men even though they are attracted to him and when his girlfriend wants him back he must make a tough decision.' popularity=8.232 poster_path='/2TcZBHcseRXLxNq2RXnKIsbKZwi.jpg' tmdb_id=159 original_language='de' vote_average=6.2 vote_count=87 homepage='' production_companies=[ProductionCompanies(id=47, logo_path='/i7Z9ot2o3N5Sa3HrF09kniFs2y8.png', name='Constantin Film', origin_country='DE'), ProductionCompanies(id=99, logo_path=None, name='Olga-Film GmbH (München)', origin_country='')] runtime=90 belongs_to_collection=None budget=0 imdb_id='tt0109255' original_title='Der bewegte Mann' release_date='1994-10-05' rev

In [33]:
display(movie_objs)

[MediaMovie(adult=False, backdrop_path='/1BMnXAAMbGN8N1xm2ZvlSVVUQiy.jpg', genres=[MediaGenres(id=35, name='Comedy'), MediaGenres(id=18, name='Drama')], genre_ids=None, overview='Der Bewegte Mann is a German comedy about a heterosexual man, Axel, who is thrown out of his girlfriends home for cheating and ends up moving in with a gay man. Axel learns the advantages of living with gay men even though they are attracted to him and when his girlfriend wants him back he must make a tough decision.', popularity=8.232, poster_path='/2TcZBHcseRXLxNq2RXnKIsbKZwi.jpg', tmdb_id=159, original_language='de', vote_average=6.2, vote_count=87, homepage='', production_companies=[ProductionCompanies(id=47, logo_path='/i7Z9ot2o3N5Sa3HrF09kniFs2y8.png', name='Constantin Film', origin_country='DE'), ProductionCompanies(id=99, logo_path=None, name='Olga-Film GmbH (München)', origin_country='')], runtime=90, belongs_to_collection=None, budget=0, imdb_id='tt0109255', original_title='Der bewegte Mann', release